In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [65]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test'] 

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_validation_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [66]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [67]:
#custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
custom_optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [68]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 9s - loss: 0.2777 - accuracy: 0.9196 - val_loss: 0.1368 - val_accuracy: 0.9575
Epoch 2/5
540/540 - 8s - loss: 0.1066 - accuracy: 0.9676 - val_loss: 0.0908 - val_accuracy: 0.9723
Epoch 3/5
540/540 - 8s - loss: 0.0703 - accuracy: 0.9779 - val_loss: 0.0695 - val_accuracy: 0.9770
Epoch 4/5
540/540 - 8s - loss: 0.0517 - accuracy: 0.9836 - val_loss: 0.0512 - val_accuracy: 0.9835
Epoch 5/5
540/540 - 8s - loss: 0.0391 - accuracy: 0.9877 - val_loss: 0.0429 - val_accuracy: 0.9860


In [69]:
test_loss, teste_accuracy = model.evaluate(test_data)

2/2 [==============================] - 1s 315ms/step - loss: 0.0711 - accuracy: 0.9780


In [70]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, teste_accuracy*100.) )

Test loss: 0.07. Test accuracy: 97.80%
